<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация комментариев

**Задача:**

Необходимо ускорить модерацию токсичных комментариев для интернет-магазина. Обучить модель машинного обучения для классификации таких комментариев.

**Данные:**

 В распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

In [1]:
import pandas as pd
import numpy as np

import re

from pandarallel import pandarallel   
from tqdm import tqdm
import spacy

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

Загрузим данные и просмотрим на них.

In [2]:
try:
    df_comments = pd.read_csv('toxic_comments.csv')
except:
    df_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(df_comments.head(5))
print('-'*50)
df_comments.info()
print('-'*50)
print('Полных дубликатов:', df_comments.duplicated().sum())
print('-'*50)
print('Пропусков:')
display(df_comments.isna().sum())
print('-'*50)
print('Соотношение в целевом признаке:')
display(df_comments.toxic.value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
--------------------------------------------------
Полных дубликатов: 0
--------------------------------------------------
Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

--------------------------------------------------
Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

Данные чистые, за исключением столбца `Unnamed: 0`. Столбец полностью повторяет индексы таблицы. Уберем этот столбец.

In [4]:
df_comments = df_comments.drop('Unnamed: 0', axis=1)

Создадим функцию для очистки текста от лишних символов.

In [5]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [6]:
df_comments['text'] = df_comments['text'].apply(clear_text) 

In [7]:
df_comments.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


Приведем слова к лемме и за одно разобьем текст на токены с помощью библиотеки `spaCy`.

In [8]:
tqdm.pandas(desc="progress")
pandarallel.initialize(progress_bar = True)

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
df_comments['text'] = df_comments['text'].parallel_apply(lambda x, nlp=nlp: " ".join([token.lemma_ for token in nlp(x)]))

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [9]:
df_comments.head()

,text,toxic
0,explanation why the edit make under my usernam...,0
1,d aww he match this background colour I m seem...,0
2,hey man I m really not try to edit war it s ju...,0
3,more I can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


Разобьем наши данные на три выборки: обучающую, валидационную и тестовую.

In [10]:
features = df_comments.drop('toxic', axis=1)
train = df_comments['toxic']

features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              train,
                                                                              test_size=.2,
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid,
                                                                            target_valid,
                                                                            test_size=.5,
                                                                            random_state=12345)

In [11]:
print(f'Размер обучающей выборки:\n{features_train.shape}\n{target_train.shape}')

Размер обучающей выборки:
(127433, 1)
(127433,)


In [12]:
print(f'Размер валидационной выборки:\n{features_valid.shape}\n{target_valid.shape}')

Размер валидационной выборки:
(15929, 1)
(15929,)


In [13]:
print(f'Размер тестовой выборки:\n{features_test.shape}\n{target_test.shape}')

Размер тестовой выборки:
(15930, 1)
(15930,)


**Вывод:**

Данные изучены и подготовлены для обучения моделей.

## Обучение

Проведем для наших признаков оценку важности слова методом TF-IDF. 

In [14]:
stopwords = nlp.Defaults.stop_words

count_tf_idf = TfidfVectorizer(stop_words=stopwords, sublinear_tf=True)

features_train = count_tf_idf.fit_transform(features_train['text'].values)
features_valid = count_tf_idf.transform(features_valid['text'].values)
features_test = count_tf_idf.transform(features_test['text'].values)

**Логистическая регрессия**

In [15]:
%%time

#train_lr:
classificator = LogisticRegression(class_weight='balanced')
    
parameters = {'C': (.1, 1, 5, 10),
              'random_state': ([12345]),
              'max_iter': ([200])}
gscv = GridSearchCV(classificator, parameters, scoring='f1', cv=5, n_jobs=-1)

gscv.fit(features_train, target_train)

mean_score = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mean_score)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#valid_lr:
predictions_valid = gscv.predict(features_valid)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()

#test_lr:
predictions_test = gscv.predict(features_test)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))

F1 логистической регрессии = 0.77
при параметрах {'C': 10, 'max_iter': 200, 'random_state': 12345}

F1 логистической регрессии на валидации = 0.77

финальный F1 логистической регрессии = 0.77
CPU times: total: 20.8 s
Wall time: 4min 5s


**CatBoost**

In [16]:
%%time

#train_cbc:
classificator = CatBoostClassifier(auto_class_weights='Balanced')
    
parameters = {'verbose': ([False]),
              'iterations': ([200]),
              'random_state':([12345])} 

gscv = GridSearchCV(classificator, parameters, scoring='f1', cv=2, n_jobs=-1)

gscv.fit(features_train, target_train)

mean_score = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mean_score)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#valid_cbc:
predictions_valid = gscv.predict(features_valid)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()

#test_cbc:
predictions_test = gscv.predict(features_test)
cbc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 CatBoostClassifier =', round(cbc_test_f1,2))

F1 CatBoostClassifier = 0.74
при параметрах {'iterations': 200, 'random_state': 12345, 'verbose': False}

F1 CatBoostClassifier на валидации = 0.75

финальный F1 CatBoostClassifier = 0.76
CPU times: total: 20min 46s
Wall time: 19min 54s


**Случайный лес**

In [17]:
%%time

#train_rfc:
classificator = RandomForestClassifier(class_weight='balanced')
    
parameters = {'n_estimators': ([300]),
              'random_state': ([12345]),
              'max_depth': ([x for x in range(11, 31, 10)])}

gscv = GridSearchCV(classificator, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mean_score = gscv.cv_results_['mean_test_score']
rfc_train_f1 = max(mean_score)

print('F1 случайного леса =', round(rfc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#valid_rfc:
predictions_valid = gscv.predict(features_valid)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации =', round(rfc_valid_f1,2))
print()

#test_rfc:
predictions_test = gscv.predict(features_test)
rfc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 случайного леса =', round(rfc_test_f1,2))

F1 случайного леса = 0.42
при параметрах {'max_depth': 21, 'n_estimators': 300, 'random_state': 12345}

F1 случайного леса на валидации = 0.42

финальный F1 случайного леса = 0.42
CPU times: total: 44.2 s
Wall time: 8min 21s


In [18]:
table = pd.DataFrame(index=['LogisticRegression',
                            'CatBoostClassifier',
                            'RandomForestClassifier'],
                     columns=['train_f1', 'valid_f1', 'test_f1'])

table['train_f1'] = f'{lr_train_f1:.2f}', f'{cbc_train_f1:.2f}', f'{rfc_train_f1:.2f}'
table['valid_f1'] = f'{lr_valid_f1:.2f}', f'{cbc_valid_f1:.2f}', f'{rfc_valid_f1:.2f}'
table['test_f1'] = f'{lr_test_f1:.2f}', f'{cbc_test_f1:.2f}', f'{rfc_test_f1:.2f}'
table

,train_f1,valid_f1,test_f1
LogisticRegression,0.77,0.77,0.77
CatBoostClassifier,0.74,0.75,0.76
RandomForestClassifier,0.42,0.42,0.42


**Вывод:**

По итогам обучения и сравнения моделей, лучшей моделью является **LogisticRegression** с метрикой `F1` в 0.77 на тестовой выборке.

## Выводы

Для нового сервиса интернет-магазина была обучена модель классификации комментариев. С помощью нового сервиса пользователи могут редактировать, дополнять описания товаров и комментировать изменения других пользователей, как в вики-сообществах. Модель определяет токсичные комментарии и отправляет их на модерацию.

Для обучения и выбора лучшей модели был предоставлен набор данных с разметкой о токсичности правок. Данные были изучены и подготовлены, а именно:
- Удаление лишнего столбца, который повторял индексы таблицы;
- Создана функция для отчистки текста от лишних символов;
- Слова были приведены к лемме (начальная форма слова);
- Данные разделены на три выборки: обучающая(train), валидационная(valid) и тестовая(test).

Для сравнения были взяты 3 модели машинного обучения:
- LogisticRegression;
- CatBoostClassifier;
- RandomForestClassifier.

После обучения и сравнения результата по значению метрики `F1`, которая должна была быть не менее 0.75, самой лучшей моделью для задачи классификации комментариев была выбрана модель **LogisticRegression** со значением метрики `F1` равному **0.77**.
